<a href="https://colab.research.google.com/github/aswinaus/ML/blob/main/ADLS_AzureSynapse_ApacheSpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession


# Initialize SparkSession with Hadoop Azure dependency
#spark = SparkSession.builder \
#    .appName("DocumentClassification") \
#    .config("spark.jars.packages", "org.apache.hadoop:hadoop-azure:3.3.1") \
#    .getOrCreate()



# Initialize SparkSession with Hadoop Azure dependency
spark = (
    SparkSession.builder
    .appName("TF-IDF-Document-Classification")
    .config("spark.jars.packages",
            "org.apache.hadoop:hadoop-azure:3.3.1,"
            "com.microsoft.azure:azure-storage:8.6.6")
    .config("spark.hadoop.fs.azure.account.key.classifydocsdlsgen2.blob.core.windows.net", "<storagekey>")
    .getOrCreate()
)



# Load millions of docs into Spark DataFrame: (id, text, label)
df = spark.read.json("abfss://docs@mydatalake.dfs.core.windows.net/documents.json")

# Tokenize
tokenizer = Tokenizer(inputCol="text", outputCol="words")

# TF step
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=100000)

# IDF step
idf = IDF(inputCol="rawFeatures", outputCol="features")

# Logistic Regression
lr = LogisticRegression(maxIter=20, regParam=0.01)

# Build pipeline
pipeline = Pipeline(stages=[tokenizer, hashingTF, idf, lr])

# Train model
model = pipeline.fit(df)

# Predict
predictions = model.transform(df)